In [1]:
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt

In [101]:
m_cluster = 250
gamma = 0.2
epsilon = 10**(-3)
delta = 0.01
k_cluster = 10
m = 250
r = 0.95
theta = np.pi/4
k_max = 250

In [3]:
def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    F = 1/(1+np.abs(f1)+np.abs(f2))
    return F

In [4]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points


In [5]:
def maximize(set_of_points):
    z = []
    z_max = 0
    F = objective_function(set_of_points.T)
    for i in range (len(set_of_points)):
        z.append(F[i])
        if z[i]>z_max:
            z_max = z[i]
            idx_min = i
    x_min = set_of_points[idx_min]
    return z_max,idx_min,x_min

In [6]:
boundaries = np.array([(-10,10),(-10,10)])
iter_points = {}
iter_points[k] = generate_points(2,m_cluster,boundaries[:,0],boundaries[:,1])
x_prime = maximize(iter_points[0])[-1]
x_prime

array([-6.71875,  0.15625])

In [7]:
min_boundaries = 10**100
for i in range (len(boundaries)):
    abs_disc = np.abs(boundaries[i,1]-boundaries[i,0])
    if abs_disc<=min_boundaries:
        min_boundaries = abs_disc
radius = min_boundaries/2
radius

10.0

In [8]:
k = 0
cluster_center,cluster_radius = {},{}
cluster_center[0],cluster_radius[0] = x_prime,radius
print(cluster_center,cluster_radius)

{0: array([-6.71875,  0.15625])} {0: 10.0}


In [9]:
# potential_cluster_center = []
# F = objective_function(iter_points[0].T)
# for i in range (m_cluster):
#     if F[i] > gamma:
#         potential_cluster_center.append(iter_points[0][i])
# len(potential_cluster_center)

In [10]:
def function_cluster(y,lendict):
    min_dist_cluster = 10**100
    for ci,cc in cluster_center.items():
        dist = np.linalg.norm(cc-y)
        if dist<=min_dist_cluster:
            xc = cc
            cluster_id = ci
            min_dist_cluster = dist
    xt = (xc + y)/2
    # print(xt,xc,y)
    Fxt = objective_function(xt)
    Fxc = objective_function(xc)
    Fy = objective_function(y)
    # print(Fxt,Fxc,Fy)
    if Fxt < Fy and Fxt < Fxc:
        cluster_center[lendict] = y
        cluster_radius[lendict] = np.linalg.norm(y-xt)
    elif Fxt > Fy and Fxt > Fxc:
        cluster_center[lendict] = y
        cluster_radius[lendict] = np.linalg.norm(y-xt)
        function_cluster(xt,lendict+1)
    elif Fy > Fxc:
        cluster_center[cluster_id] = y

    cluster_radius[cluster_id] =  np.linalg.norm(y-xt)
    # return cluster_center,cluster_radius


In [11]:
# for i in range (len(potential_cluster_center)):
#     print(f'Titik ke-{i}')
#     lendict = len(list(cluster_center.keys()))
#     function_cluster(potential_cluster_center[i],lendict)
#     print(cluster_center,cluster_radius)
#     print('')

In [12]:
import spiralopt as spo
Sn = r*spo.generate_Rn(dim=2,theta=theta)
k=0
while k<k_cluster:
    potential_cluster_center = []
    F = objective_function(iter_points[k].T)
    for i in range (m_cluster):
        if F[i] > gamma:
            potential_cluster_center.append(iter_points[k][i])
    len(potential_cluster_center)
    # print('')
    for i in range (len(potential_cluster_center)):
        # print(f'Titik ke-{i}')
        lendict = len(list(cluster_center.keys()))
        function_cluster(potential_cluster_center[i],lendict)
        # print(cluster_center,cluster_radius)
        # print('')
    iter_points[k+1]=spo.update_point(iter_points[k],Sn,dim=2)
    k+=1
cluster_center,cluster_radius

({0: array([-6.71875,  0.15625]),
  1: array([-1.30078125,  0.82421875]),
  2: array([-0.3125,  6.5625]),
  3: array([-0.74414062,  1.15820313]),
  4: array([0.234375, 6.640625]),
  5: array([-0.0390625,  6.6015625]),
  6: array([0.09765625, 6.62109375]),
  7: array([0.36914063, 0.93554688]),
  8: array([0.17578125, 5.29296875]),
  9: array([-3.12285156, -0.05527344]),
  10: array([-0.55859375,  1.86328125]),
  11: array([0.734375  , 0.68554688]),
  12: array([-1.00390625,  0.52734375]),
  13: array([0.24609375, 0.62109375]),
  14: array([-0.3359375,  3.2734375]),
  15: array([-3.38725586e+00, -2.39257813e-03]),
  16: array([0.03515625, 2.90234375]),
  17: array([0.14648438, 2.04882813]),
  18: array([-0.20605469,  1.95605469]),
  19: array([-0.02978516,  2.00244141]),
  20: array([-0.11791992,  1.97924805]),
  21: array([-0.08486938,  1.98794556]),
  22: array([-0.09588623,  1.98504639]),
  23: array([0.88867188, 0.41601563]),
  24: array([-0.96679687, -0.40039063]),
  25: array([-5.5

In [30]:
def cluster_boundaries(center_point,radius):
    return np.array([[center_point[i]-radius,center_point[i]+radius]for i in range (len(center_point))])
cenpo = cluster_center[20]
radpo = cluster_radius[20]
boundy = cluster_boundaries(cenpo,radpo)
lowpo = boundy[:,0]
highpo = boundy[:,1]

In [34]:
roots = []
roots_values = []
for i in range (len(cluster_center)):
    bound = cluster_boundaries(cluster_center[i],cluster_radius[i])
    lp = bound[:,0]
    hp = bound[:,1]
    root,value = spo.SpiralOpt(lp,hp,dim=2,npoint=m,r = r,theta=theta, iter_max=k_max, error_max = 10**(-5),random=0, show_err=False, show_objective_function=False)
    roots.append(root)
    roots_values.append(value)
print(roots)
print(roots_values)

[[-6.656672674453279, 0.14831858727959457], [-0.9589305978247163, 1.0410743128670283], [-0.15528773088258727, 6.439839302128859], [-0.9321160283519141, 1.0678776270017354], [0.14764259808777927, 6.520644176116848], [0.1633174021182439, 6.1229000785634575], [0.14493133180927578, 6.572129368692184], [0.43485221950282715, 0.9603962193806436], [0.16670462285911938, 5.610939578138676], [-3.1817110413387693, -0.0014676136105186911], [-0.46828353336980055, 1.5131108373542634], [0.6671212598392549, 0.6901045084974029], [-1.1268863485642973, 0.8730607104833668], [0.6283686024645031, 0.7301404023296069], [-0.00306627878787726, 3.09971253966771], [-3.216408708705316, 0.036061416704750444], [0.04154869504936977, 3.0506653322446526], [-0.011573926546033078, 1.8879218730972287], [-0.11563687952736819, 1.8656368795273692], [-0.05719334309143927, 1.958659421768079], [-0.09532030800283255, 1.9566484330028342], [-0.0735730305474649, 1.9766492024224667], [-0.08458987625058992, 1.9737500325005917], [0.667

In [37]:
print(len(roots))

97


In [41]:
for i in roots:
    print(1-objective_function(i))

0.1832736608099791
0.050597463478999405
5.774534414648702e-05
1.4343704123565004e-05
0.2722111816638412
0.0004940696502676589
0.29520198056895064
0.28244933314447573
0.32823045848691046
0.49978959864763783
0.4209524570975004
3.6940884987624045e-06
0.2787431426812035
0.06934960405588309
0.499771767458165
0.5031719483718227
0.5036901389762207
0.5250159496137601
0.5171136669700039
0.5348499494478843
0.535069065879148
0.5378806527130705
0.5376730637079421
0.0013215621576269099
0.585472690521031
0.3412849912279383
0.4717248055304706
0.475310958129457
0.03986862023859472
0.1043737036857989
0.49111633430553425
0.04949520077355751
0.01272694752031589
0.3446971779638922
0.28372854667170644
0.4268128669154131
0.2038545734994025
0.4687526881408448
0.47162772391309826
0.4185780832646069
0.4903238676509236
0.243745858200493
0.15196377856433185
0.11285800335939122
0.3573475805776818
0.49653831124769565
0.37599152842747996
0.26992984465514824
0.49360100588247946
0.47132264424406034
0.4712447073212290

In [77]:
eligible_roots = np.array([x for x in roots if 1-objective_function(x)<epsilon])
eligible_roots

array([[-0.15528773,  6.4398393 ],
       [-0.93211603,  1.06787763],
       [ 0.1633174 ,  6.12290008],
       [ 0.66712126,  0.69010451],
       [-6.43716217,  0.15534518],
       [-6.11711888, -0.16347392],
       [-6.11710807, -0.16347572]])

In [103]:
np.linalg.norm(np.array([-6.43716217  ,0.15534518])-np.array([-6.11711888,-0.16347392]))<delta

False

In [109]:
duplicated_roots = []
for i in range(len(eligible_roots)):
    for j in range (i+1,len(eligible_roots)):
        if np.linalg.norm(eligible_roots[i]-eligible_roots[j])<delta:
            duplicated_roots.append([eligible_roots[i],eligible_roots[j]])
duplicated_roots = np.unique(duplicated_roots,axis=0)
print(duplicated_roots)
deselected_duplicated_roots = []
for i in range (len(duplicated_roots)):
    root_a = objective_function(duplicated_roots[i][0])
    root_b = objective_function(duplicated_roots[i][1])
    if root_a>root_b:
        deselected_duplicated_roots.append(duplicated_roots[i][1])
    else:
        deselected_duplicated_roots.append(duplicated_roots[i][0])
final_root = eligible_roots[~np.all(eligible_roots == deselected_duplicated_roots, axis=1)]
final_root

[[[-6.11711888 -0.16347392]
  [-6.11710807 -0.16347572]]]


array([[-0.15528773,  6.4398393 ],
       [-0.93211603,  1.06787763],
       [ 0.1633174 ,  6.12290008],
       [ 0.66712126,  0.69010451],
       [-6.43716217,  0.15534518],
       [-6.11710807, -0.16347572]])

In [13]:
# """Visualization"""
# fig, ax = plt.subplots()
# for center,radius in zip(cluster_center.values(),cluster_radius.values()):
#     circle = plt.Circle(center, radius, fill=False, linestyle='dotted', edgecolor='b')
#     ax.add_artist(circle)

# # Set axis limits
# ax.set_xlim(-10, 10)
# ax.set_ylim(-10, 10)

# # # Add labels (optional)
# # for i, center in cluster_center.items():
# #     ax.text(center[0], center[1], f'Cluster {i+1}', ha='center', va='bottom')

# # Add a title and labels (optional)
# ax.set_title('Cluster Visualization')
# ax.set_xlabel('X-axis')
# ax.set_ylabel('Y-axis')

# # Show the plot
# plt.gca().set_aspect('equal', adjustable='box')  # Make the aspect ratio equal
# plt.grid(True)
# plt.show()
